In [ ]:
## Open Sample  file instead of present file.
from PyHiLo import *
with open ("/home/youssef15015/older_files/outgoing/63617L7H7.pickle") as f:
    hilo=pickle.load(f)

In [1]:
## Functions used to collect channel and monitor charges.
def collect(telID,chanID):

    hiGainEvts=np.where(hilo.hiLo[telID][chanID][:]==0)
    lowGainEvts=np.where(hilo.hiLo[telID][chanID][:]==1)

    mon = hilo.meanOfMedian
    Q = hilo.allCharge

    mon1 = mon[telID]
    Q1 = Q[telID][chanID]


    mon1_Hi = mon1[hiGainEvts]
    mon1_Lo = mon1[lowGainEvts]
    Q1_Hi = Q1[hiGainEvts]
    Q1_Lo = Q1[lowGainEvts]
    return mon1_Hi,mon1_Lo, Q1_Hi,Q1_Lo

##Montior charge as a function of the high gain charge values.
def f_Hi(charge,m_Hi,c):
    return m_Hi*charge+c

##Montior charge as a function of the low gain charge values.
def f_Lo(charge,m_Lo,c):
    return m_Lo*charge+c

#This defines a piecewise function as a function of inputted 2d data (returning the monitor charge)
def Mon(data,m_Hi,m_Lo,c):
    result1 = f_Hi(data[0],m_Hi,c)
    result2 = f_Lo(data[1],m_Lo,c)
    return np.append(result1,result2)

def cleanup(value,min,max):
    value = np.array(value)
    value = value[np.where(value>min)]
    value = value[np.where(value<max)]
    return value

In [ ]:
mask=[[],[],[],[]]
#mask[0] = [49,50,75,76,107,110,125,232]
#mask[1] = [81,213,230]
#mask[2] = [37,38,54,65,123,128,159,178,180,188,213,230]
#mask[3] = [99,150]
avg = [[],[],[],[]]
for tel in range(4):
## Use (250,499) for inner hi gain. 
    for i in range(249):
        if i in mask[tel]:
            continue
        params = collect(tel,i)
        data = [params[2],params[3]]
        y = np.append(params[0],params[1])
        ## curve_fit defined in PyHiLo (imported from scipy.optimize)
        ## curve_fit(f(x),x,y), returns parameters.
        Fit = curve_fit(Mon,data,y)
        ## Calculate the 1 sigma of error from curve_fit's cross correlation matrix
        cov=np.sqrt(np.diag(Fit[1]))
        ## Error of inverse Higain multiplier
        sig_1 = cov[0]/m2**2
        sig_2 = cov[1]
        sig = (m1/m2)*((m2*sig_1)**2.0+(sig_2/m1)**2.0)**(0.5)
        print(Fit[0][1]/Fit[0][0],sig,i)
        #Print(inverse of low gain divided by inverse of High gain multiplier) and Error
        print(Fit[0][1]/Fit[0][0], sig,i)
        avg[tel].append(Fit[0][1]/Fit[0][0])
print(np.mean(avg[0]))
print(np.mean(avg[1]))
print(np.mean(avg[2]))
print(np.mean(avg[3]))

In [ ]:
#Plots
#Please use these to inspect wonky curves.
telID=0
for i in range(0,249):
    mon1_Hi,mon1_Lo,Q1_Hi,Q1_Lo = collect(telID,i)
    
    
    data = [Q1_Hi,Q1_Lo]
    y = np.append(mon1_Hi ,mon1_Lo)
    m = curve_fit(Gain,data,y)
    m1 = m[0][1]
    m2 = m[0][0]  
    cov=np.sqrt(np.diag(m[1]))
    sig_1 = cov[0]/m2**2
    sig_2 = cov[1]
    sig = (m1/m2)*((m2*sig_1)**2.0+(sig_2/m1)**2.0)**(0.5)
    
    print(m)
    
    plt.plot(mon1_Lo,Q1_Lo*m1/m2,'r.',label = r'Low Gain * %f $\pm$  %f' % (m1/m2,sig))
    plt.plot(mon1_Hi,Q1_Hi,'g.', label = 'HiGain')
    plt.plot(mon1_Lo,mon1_Lo/m2,'b',label='HiGain Slope')
    plt.plot(mon1_Hi,mon1_Hi/m2,'b')
    plt.legend()
    plt.title('Monitor vs Channel Charge. Tel %d Channel %d' % (telID+1,i))
    plt.show()